In [1]:
import pandas as pd
import numpy as np
import gnupg

In [6]:
# Create a GPG object
#gpg_binary_path = "C:\\Program Files (x86)\\Gpg4win\\bin\\"
gpg = gnupg.GPG()
import_result = gpg.import_keys('public.key')

In [7]:
# Load the data (replace 'your_file.csv' with the actual file path)
dataframe = pd.read_csv('weather_2023-12.csv', header=None)

In [8]:
# Create a new column for decrypted values
dataframe[3] = np.nan  # Initialize with missing values

In [9]:
# Decrypt the encrypted values in the 3rd column (if applicable)
def is_encrypted(value):
  return value.startswith("-----BEGIN PGP MESSAGE-----")

def decrypt_value(value):
    if is_encrypted(value):  # Replace with your logic to identify encrypted values
        try:
            decrypted_data = gpg.decrypt(value.encode())
            return decrypted_data.data.decode()
        except Exception as e:
            print(f"Error decrypting value: {e}")
            return value  # Or handle the error as needed
    else:
        return value

In [11]:
# Decrypt and populate the new column
for i in range(len(dataframe)):
    value = dataframe.iloc[i, 2]
    if pd.notna(value) and is_encrypted(value):
        try:
            decrypted_data = gpg.decrypt(value.encode())
            dataframe.iloc[i, 3] = decrypted_data.data.decode()
        except Exception as e:
            print(f"Error decrypting value: {e}")

In [13]:
# Get cleaned (ignoring NaN values) maximum and minimum temperatures
max_temp_cleaned = dataframe.loc[dataframe[3].str.startswith('TA_PT1H_MAX') & dataframe[3].notna()]
min_temp_cleaned = dataframe.loc[dataframe[3].str.startswith('TA_PT1H_MIN') & dataframe[3].notna()]

In [14]:
# Calculate the difference between max and min temperatures
temperature_difference = max_temp_cleaned - min_temp_cleaned

In [15]:
# Compute average difference and standard deviation
avg_difference = temperature_difference.mean()
std_deviation = temperature_difference.std()

In [16]:
# Define the range for the condition
lower_bound = avg_difference - 0.6 * std_deviation
upper_bound = avg_difference + 0.6 * std_deviation

In [17]:
# Filter observations that satisfy the condition
satisfying_observations = dataframe[
    (temperature_difference >= lower_bound) &
    (temperature_difference <= upper_bound)
]

In [18]:
# Calculate the percentage
percentage_satisfying = (len(satisfying_observations) / len(dataframe)) * 100

In [19]:
# Write the answer to the specified file
with open('exrc-04-answer.txt', 'w') as result_file:
    result_file.write(f"{percentage_satisfying:.2f}")

print(f"Percentage of hourly observations satisfying the condition: {percentage_satisfying:.2f}%")

Percentage of hourly observations satisfying the condition: 100.00%


### **Code from Farhad**

In [ ]:
import pandas as pd
import numpy as np
import gnupg

gpg_binary_path = 'C:\\Program Files (x86)\\gnupg\\bin\\gpg.exe'
gpg = gnupg.GPG(binary=gpg_binary_path)


import_result = gpg.import_keys('public.key')

# Load the data (replace 'your_file.csv' with the actual file path)
dataframe = pd.read_csv('weather_2023-12.csv', header=None)

# Create a new column for decrypted values
dataframe[3] = np.nan  # Initialize with missing values

# Decrypt the encrypted values in the 3rd column (if applicable)
def is_encrypted(value):
  return value.startswith("-----BEGIN PGP MESSAGE-----")

def decrypt_value(value):
    if is_encrypted(value):  # Replace with your logic to identify encrypted values
        try:
            decrypted_data = gpg.decrypt(value.encode())
            return decrypted_data.data.decode()
        except Exception as e:
            print(f"Error decrypting value: {e}")
            return value  # Or handle the error as needed
    else:
        return value
    
# Decrypt and populate the new column
for i in range(len(dataframe)):
    value = dataframe.iloc[i, 2]
    if is_encrypted(value):
        try:
            decrypted_data = gpg.decrypt(value.encode())
            dataframe.iloc[i, 3] = decrypted_data.data.decode()
        except Exception as e:
            print(f"Error decrypting value: {e}")    

# Get cleaned (ignoring NaN values) maximum and minimum temperatures
max_temp_cleaned = dataframe[dataframe[3].str.startswith('TA_PT1H_MAX')].dropna() #weather_data['TA_PT1H_MAX'].dropna()
min_temp_cleaned = dataframe[dataframe[3].str.startswith('TA_PT1H_MIN')].dropna() #weather_data['TA_PT1H_MIN'].dropna()

# Calculate the difference between max and min temperatures
temperature_difference = max_temp_cleaned - min_temp_cleaned

# Compute average difference and standard deviation
avg_difference = temperature_difference.mean()
std_deviation = temperature_difference.std()

# Define the range for the condition
lower_bound = avg_difference - 0.6 * std_deviation
upper_bound = avg_difference + 0.6 * std_deviation

# Filter observations that satisfy the condition
satisfying_observations = dataframe[
    (temperature_difference >= lower_bound) &
    (temperature_difference <= upper_bound)
]

# Calculate the percentage
percentage_satisfying = (len(satisfying_observations) / len(dataframe)) * 100

# Write the answer to the specified file
with open('exrc-04-answer.txt', 'w') as result_file:
    result_file.write(f"{percentage_satisfying:.2f}")

print(f"Percentage of hourly observations satisfying the condition: {percentage_satisfying:.2f}%")